<a href="https://colab.research.google.com/github/cw118/codeml2024/blob/main/air_canada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf

### Air Canada

In [3]:
baseline = pd.read_csv("baseline.csv")

baseline.head()

,id,ticket_id,od,flight_departure_datetime,purchase_datetime,trip_type,branded_fare,number_of_pax,ADVS_price,PREF_price,ADVS_capacity,PREF_capacity,ADVS_inventory,PREF_inventory,choice
0,0,4157258036156055,BBBDDD,2024-01-01 03:00,2023-07-23 15:08,0,2,3,48,86,50,10,50,10,NaN
1,1,"7,18E+15",BBBDDD,2024-01-01 03:00,2023-07-26 11:44,1,2,1,37,82,50,10,50,7,NaN
2,5,3193079964098903,BBBDDD,2024-01-01 03:00,2023-08-29 10:30,0,0,1,50,100,50,10,50,4,NaN
3,6,7188323188824055,BBBDDD,2024-01-01 03:00,2023-09-09 17:14,0,1,2,34,71,50,10,50,4,NaN
4,9,8613619724770241,BBBDDD,2024-01-01 03:00,2023-09-29 10:01,0,3,3,40,85,50,10,48,3,NaN


In [4]:
participant = pd.read_csv("participant_data.csv")

participant.head()

,id,ticket_id,od,flight_departure_datetime,purchase_datetime,trip_type,branded_fare,number_of_pax,ADVS_price,PREF_price,ADVS_capacity,PREF_capacity,ADVS_inventory,PREF_inventory,choice
0,0,4157258036156055,BBBDDD,2024-01-01 03:00,2023-07-23 15:08,0,2,3,48,86,50,10,50,10,NaN
1,1,"7,18E+15",BBBDDD,2024-01-01 03:00,2023-07-26 11:44,1,2,1,37,82,50,10,50,7,NaN
2,2,3353015593651433,BBBDDD,2024-01-01 03:00,2023-08-03 06:03,0,1,2,35,80,50,10,50,7,pref
3,3,5763957877920625,BBBDDD,2024-01-01 03:00,2023-08-13 17:41,0,3,1,36,68,50,10,50,5,nochoice
4,4,1095418927576125,BBBDDD,2024-01-01 03:00,2023-08-20 22:04,0,3,1,57,70,50,10,50,5,pref


In [5]:
df_pred = baseline[baseline['choice'].notna()]
df = participant[participant['choice'].notna()]

Train on participant data, predict on baseline data.

In [6]:
df = df.drop(columns=['ticket_id', 'od'])

In [7]:
df.head(15)

,id,flight_departure_datetime,purchase_datetime,trip_type,branded_fare,number_of_pax,ADVS_price,PREF_price,ADVS_capacity,PREF_capacity,ADVS_inventory,PREF_inventory,choice
2,2,2024-01-01 03:00,2023-08-03 06:03,0,1,2,35,80,50,10,50,7,pref
3,3,2024-01-01 03:00,2023-08-13 17:41,0,3,1,36,68,50,10,50,5,nochoice
4,4,2024-01-01 03:00,2023-08-20 22:04,0,3,1,57,70,50,10,50,5,pref
7,7,2024-01-01 03:00,2023-09-17 22:16,1,3,1,41,76,50,10,48,4,pref
8,8,2024-01-01 03:00,2023-09-25 00:53,1,0,3,44,81,50,10,48,3,nochoice
10,10,2024-01-01 03:00,2023-09-30 00:35,0,1,1,38,81,50,10,48,0,advs
12,12,2024-01-01 03:00,2023-10-19 00:59,0,0,1,44,75,50,10,47,0,advs
13,13,2024-01-01 03:00,2023-10-19 11:20,0,1,1,38,73,50,10,46,0,nochoice
14,14,2024-01-01 03:00,2023-10-24 00:48,0,0,2,43,81,50,10,46,0,nochoice
15,15,2024-01-01 03:00,2023-11-06 20:17,0,1,1,25,76,50,10,46,0,nochoice


Inventory probably not helpful, capacity (constant values) likely not either.

In [8]:
df = df.drop(columns=['ADVS_capacity', 'PREF_capacity'])

In [9]:
import matplotlib.pyplot as plt

Convert dates into timestamps (seconds or ms).

In [10]:
df['flight_departure_datetime'] = pd.to_datetime[df['flight_departure_datetime']]
df['flight_departure_datetime'] = df['flight_departure_datetime'].type(int64) // 10**9

df['flight_departure_datetime']

TypeError: 'function' object is not subscriptable

Turning choice into int (0=nochoice, 1=pref, 2=advs) and checking number of occurrences for each

In [14]:
df.loc[df["choice"] == "nochoice", "choice"] = 0
df.loc[df["choice"] == "pref", "choice"] = 1
df.loc[df["choice"] == "advs", "choice"] = 2

print(df['choice'].value_counts())

df.head(15)

choice
0    130886
2     33331
1     28281
Name: count, dtype: int64


,id,flight_departure_datetime,purchase_datetime,trip_type,branded_fare,number_of_pax,ADVS_price,PREF_price,ADVS_inventory,PREF_inventory,choice
2,2,2024-01-01 03:00,2023-08-03 06:03,0,1,2,35,80,50,7,1
3,3,2024-01-01 03:00,2023-08-13 17:41,0,3,1,36,68,50,5,0
4,4,2024-01-01 03:00,2023-08-20 22:04,0,3,1,57,70,50,5,1
7,7,2024-01-01 03:00,2023-09-17 22:16,1,3,1,41,76,48,4,1
8,8,2024-01-01 03:00,2023-09-25 00:53,1,0,3,44,81,48,3,0
10,10,2024-01-01 03:00,2023-09-30 00:35,0,1,1,38,81,48,0,2
12,12,2024-01-01 03:00,2023-10-19 00:59,0,0,1,44,75,47,0,2
13,13,2024-01-01 03:00,2023-10-19 11:20,0,1,1,38,73,46,0,0
14,14,2024-01-01 03:00,2023-10-24 00:48,0,0,2,43,81,46,0,0
15,15,2024-01-01 03:00,2023-11-06 20:17,0,1,1,25,76,46,0,0
